In [1]:
import pandas as pd
import os.path
import re
import sqlite3 as sql

In [3]:

df = pd.read_csv('HH_api/data/result.csv', encoding='cp1251')
df = df.drop(columns='Unnamed: 0')
df['lower_description'] = df.description.str.lower()
df['lower_area'] = df['area_str'].str.lower()
df['lower_key_skills'] = df['key_skills'].str.lower()

In [87]:
# df_res = df.head(0)
# # df_res
# # (df['description'].iloc[0]).find(' хочешь ')
# # ls_and = ['data', 'аналитик', 'данных', 'analyst']
# ls_or = [' da ', ' da,', ' da.', 'data analyst', 'аналитик данных']
#
# search_string = ''
# for phrase in  ls_or:
#     if phrase != ls_or[-1]:
#         search_string = search_string + '(.*'+phrase+'.*)|'
#     else:
#         search_string = search_string + '(.*'+phrase+'.*)'
# print(search_string)
#
#
# for i in range(len(df)):
#     temp_index = 9999999
#     # if re.search(search_string, str.lower(df['description'].iloc[i])) != None \
#     if (len(re.findall(search_string, str.lower(df['description'].iloc[i]))) > 0) \
#             and (str.lower(df['area_str'].iloc[0]) == 'москва'):
#         temp_index = i
#     if temp_index != 9999999:
#         df_res = df_res.append(df.iloc[i])
#
# df_res

(.* da .*)|(.* da,.*)|(.* da..*)|(.*data analyst.*)|(.*аналитик данных.*)


,id,description,key_skills,schedule,experience,code,department,employment,archived,name,...,published_at,has_test,test,salary_str,area_str,employer_str,type_str,hr,hr_email,hr_phones


In [5]:
sql_db = sql.connect('HH_api/data/sql_res.db')
sql_con = sql_db.cursor()


In [9]:
# sql_con.execute('drop table vacs')

df.to_sql('vacs', sql_db, if_exists='replace', index=False)

pd.DataFrame(sql_con.execute('''
SELECT
	'https://hh.ru/vacancy/'||v.id
	, v.name
	, key_skills
	, v.experience
	, v.department
	, v.salary_str
	, v.hr
	, hr_email
	, hr_phones
FROM
	vacs v
WHERE
	1=1
	AND (v.lower_description LIKE '%data analyst%'
		OR v.lower_description LIKE '%аналитик данных%'
		OR v.description  LIKE ('% DA %')
		OR v.description  LIKE ('% DA.%')
		OR v.description  LIKE ('% DA,%')
		OR v.description  LIKE ('DA,%')
		OR v.description  LIKE ('DA.%')
		OR v.description  LIKE ('DA %'))
	AND v.lower_area = 'москва'
	AND v.lower_description LIKE '%python%'
ORDER BY
	date(created_at, 'DD-MM-YYYY')
''')).to_csv('HH_api/data/filtred.csv')

In [ ]:
sql_db.close()